In [7]:
import requests
import re
import json
from IPython.display import display, HTML
import pandas as pd
from bs4 import BeautifulSoup
# get updated baseball savant percentile rankings

endpoints = {
    'batters_percentile':'https://baseballsavant.mlb.com/leaderboard/percentile-rankings?type=batter&team=',
    'pitchers_percentile':'https://baseballsavant.mlb.com/leaderboard/percentile-rankings?type=pitcher&year=2022&team=',
    'batters_custom':"https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&sort=4&sortDir=desc&min=10&selections=player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,b_rbi,b_lob,b_total_bases,r_total_caught_stealing,r_total_stolen_base,b_ab_scoring,b_ball,b_called_strike,b_catcher_interf,b_foul,b_foul_tip,b_game,b_gnd_into_dp,b_gnd_into_tp,b_gnd_rule_double,b_hit_by_pitch,b_hit_ground,b_hit_fly,b_hit_into_play,b_hit_line_drive,b_hit_popup,b_out_fly,b_out_ground,b_out_line_drive,b_out_popup,b_intent_ball,b_intent_walk,b_interference,b_pinch_hit,b_pinch_run,b_pitchout,b_played_dh,b_sac_bunt,b_sac_fly,b_swinging_strike,r_caught_stealing_2b,r_caught_stealing_3b,r_caught_stealing_home,r_defensive_indiff,r_interference,r_pickoff_1b,r_pickoff_2b,r_pickoff_3b,r_run,r_stolen_base_2b,r_stolen_base_3b,r_stolen_base_home,b_total_ball,b_total_sacrifices,b_total_strike,b_total_swinging_strike,b_total_pitches,r_stolen_base_pct,r_total_pickoff,b_reached_on_error,b_walkoff,b_reached_on_int,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pop_2b_sba_count,pop_2b_sba,pop_2b_sb,pop_2b_cs,pop_3b_sba_count,pop_3b_sba,pop_3b_sb,pop_3b_cs,exchange_2b_3b_sba,maxeff_arm_2b_3b_sba,n_outs_above_average,n_fieldout_5stars,n_opp_5stars,n_5star_percent,n_fieldout_4stars,n_opp_4stars,n_4star_percent,n_fieldout_4stars,n_fieldout_3stars,n_opp_3stars,n_3star_percent,n_fieldout_3stars,n_opp_2stars,n_2star_percent,n_fieldout_2stars,n_opp_2stars,n_1star_percent,n_fieldout_1stars,n_opp_1stars,n_1star_percent,rel_league_reaction_distance,rel_league_burst_distance,rel_league_routing_distance,rel_league_bootup_distance,f_bootup_distance,n_bolts,hp_to_1b,sprint_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm",
    'pitchers_custom':"https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=pitcher&filter=&sort=4&sortDir=asc&min=10&selections=player_age,p_game,p_formatted_ip,p_total_pa,p_ab,p_total_hits,p_single,p_double,p_triple,p_home_run,p_strikeout,p_walk,p_k_percent,p_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,p_earned_run,p_run,p_save,p_blown_save,p_out,p_win,p_loss,p_wild_pitch,p_balk,p_shutout,p_era,p_opp_batting_avg,p_opp_on_base_avg,p_total_stolen_base,p_pickoff_attempt_1b,p_pickoff_attempt_2b,p_pickoff_attempt_3b,p_pickoff_1b,p_pickoff_2b,p_pickoff_3b,p_lob,p_rbi,p_stolen_base_2b,p_stolen_base_3b,p_stolen_base_home,p_quality_start,p_walkoff,p_run_support,p_ab_scoring,p_automatic_ball,p_ball,p_called_strike,p_catcher_interf,p_caught_stealing_2b,p_caught_stealing_3b,p_caught_stealing_home,p_complete_game,p_defensive_indiff,p_foul,p_foul_tip,p_game_finished,p_game_in_relief,p_gnd_into_dp,p_gnd_into_tp,p_gnd_rule_double,p_hit_by_pitch,p_hit_fly,p_hit_ground,p_hit_line_drive,p_hit_into_play,p_hit_scoring,p_hold,p_intent_ball,p_intent_walk,p_missed_bunt,p_out_fly,p_out_ground,p_out_line_drive,p_passed_ball,p_pickoff_error_1b,p_pickoff_error_2b,p_pickoff_error_3b,p_pitchout,p_relief_no_out,p_sac_bunt,p_sac_fly,p_starting_p,p_swinging_strike,p_unearned_run,p_total_ball,p_total_bases,p_total_caught_stealing,p_total_pickoff,p_total_pickoff_attempt,p_total_pickoff_error,p_total_pitches,p_total_sacrifices,p_total_strike,p_total_swinging_strike,p_inh_runner,p_inh_runner_scored,p_beq_runner,p_beq_runner_scored,p_reached_on_error,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pitch_hand,n,n_ff_formatted,ff_avg_speed,ff_avg_spin,ff_avg_break_x,ff_avg_break_z,ff_avg_break,ff_range_speed,n_sl_formatted,sl_avg_speed,sl_avg_spin,sl_avg_break_x,sl_avg_break_z,sl_avg_break,sl_range_speed,n_ch_formatted,ch_avg_speed,ch_avg_spin,ch_avg_break_x,ch_avg_break_z,ch_avg_break,ch_range_speed,n_cukc_formatted,cu_avg_speed,cu_avg_spin,cu_avg_break_x,cu_avg_break_z,cu_avg_break,cu_range_speed,n_sift_formatted,si_avg_speed,si_avg_spin,si_avg_break_x,si_avg_break_z,si_avg_break,si_range_speed,n_fc_formatted,fc_avg_speed,fc_avg_spin,fc_avg_break_x,fc_avg_break_z,fc_avg_break,fc_range_speed,n_fs_formatted,fs_avg_speed,fs_avg_spin,fs_avg_break_x,fs_avg_break_z,fs_avg_break,fs_range_speed,n_kn_formatted,kn_avg_speed,kn_avg_spin,kn_avg_break_x,kn_avg_break_z,kn_avg_break,kn_range_speed,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break_z,fastball_avg_break,fastball_range_speed,n_breaking_formatted,breaking_avg_speed,breaking_avg_spin,breaking_avg_break_x,breaking_avg_break_z,breaking_avg_break,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm"
}

with open('../housekeeping/baseball_savant_endpoints.txt', 'w') as file:
    file.write(json.dumps(endpoints))

position_mapping = {
    1:'P',
    2:'C',
    3:'1B',
    4:'2B',
    5:'3B',
    6:'SS',
    7:'LF',
    8:'CF',
    9:'RF',
    10:'DH',
    11:'UTIL'
}

def get_baseball_savant_leaderboard(endpoint):

    response = requests.get(endpoint)
    soup = BeautifulSoup(response.text, 'html.parser')

    # find leaderboard data (should only be one element.)
    script = str(soup.find_all("div", {"class":"article-template"})[0])

    # use regex and put everything within brackets to create a pseudo-list
    # then load this as json.
    leaderboard = json.loads('[' + re.findall('\[(.*?)\]', script)[0] + ']')

    df = pd.DataFrame(leaderboard)

    return df

# batters_percentiles = get_baseball_savant_leaderboard(endpoints['batters_percentile'])
pitchers_percentiles = get_baseball_savant_leaderboard(endpoints['pitchers_percentile'])
# batter_custom_leaderboard  = get_baseball_savant_leaderboard(endpoints['batters_custom'])
# pitcher_custom_leaderboard = get_baseball_savant_leaderboard(endpoints['pitchers_custom'])

In [8]:
# custom leaderboard endpoint

batters_endpoint = "https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&sort=4&sortDir=desc&min=10&selections=player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,b_rbi,b_lob,b_total_bases,r_total_caught_stealing,r_total_stolen_base,b_ab_scoring,b_ball,b_called_strike,b_catcher_interf,b_foul,b_foul_tip,b_game,b_gnd_into_dp,b_gnd_into_tp,b_gnd_rule_double,b_hit_by_pitch,b_hit_ground,b_hit_fly,b_hit_into_play,b_hit_line_drive,b_hit_popup,b_out_fly,b_out_ground,b_out_line_drive,b_out_popup,b_intent_ball,b_intent_walk,b_interference,b_pinch_hit,b_pinch_run,b_pitchout,b_played_dh,b_sac_bunt,b_sac_fly,b_swinging_strike,r_caught_stealing_2b,r_caught_stealing_3b,r_caught_stealing_home,r_defensive_indiff,r_interference,r_pickoff_1b,r_pickoff_2b,r_pickoff_3b,r_run,r_stolen_base_2b,r_stolen_base_3b,r_stolen_base_home,b_total_ball,b_total_sacrifices,b_total_strike,b_total_swinging_strike,b_total_pitches,r_stolen_base_pct,r_total_pickoff,b_reached_on_error,b_walkoff,b_reached_on_int,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pop_2b_sba_count,pop_2b_sba,pop_2b_sb,pop_2b_cs,pop_3b_sba_count,pop_3b_sba,pop_3b_sb,pop_3b_cs,exchange_2b_3b_sba,maxeff_arm_2b_3b_sba,n_outs_above_average,n_fieldout_5stars,n_opp_5stars,n_5star_percent,n_fieldout_4stars,n_opp_4stars,n_4star_percent,n_fieldout_4stars,n_fieldout_3stars,n_opp_3stars,n_3star_percent,n_fieldout_3stars,n_opp_2stars,n_2star_percent,n_fieldout_2stars,n_opp_2stars,n_1star_percent,n_fieldout_1stars,n_opp_1stars,n_1star_percent,rel_league_reaction_distance,rel_league_burst_distance,rel_league_routing_distance,rel_league_bootup_distance,f_bootup_distance,n_bolts,hp_to_1b,sprint_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm"
pitcher_endpoint = "https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=pitcher&filter=&sort=4&sortDir=asc&min=10&selections=player_age,p_game,p_formatted_ip,p_total_pa,p_ab,p_total_hits,p_single,p_double,p_triple,p_home_run,p_strikeout,p_walk,p_k_percent,p_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,p_earned_run,p_run,p_save,p_blown_save,p_out,p_win,p_loss,p_wild_pitch,p_balk,p_shutout,p_era,p_opp_batting_avg,p_opp_on_base_avg,p_total_stolen_base,p_pickoff_attempt_1b,p_pickoff_attempt_2b,p_pickoff_attempt_3b,p_pickoff_1b,p_pickoff_2b,p_pickoff_3b,p_lob,p_rbi,p_stolen_base_2b,p_stolen_base_3b,p_stolen_base_home,p_quality_start,p_walkoff,p_run_support,p_ab_scoring,p_automatic_ball,p_ball,p_called_strike,p_catcher_interf,p_caught_stealing_2b,p_caught_stealing_3b,p_caught_stealing_home,p_complete_game,p_defensive_indiff,p_foul,p_foul_tip,p_game_finished,p_game_in_relief,p_gnd_into_dp,p_gnd_into_tp,p_gnd_rule_double,p_hit_by_pitch,p_hit_fly,p_hit_ground,p_hit_line_drive,p_hit_into_play,p_hit_scoring,p_hold,p_intent_ball,p_intent_walk,p_missed_bunt,p_out_fly,p_out_ground,p_out_line_drive,p_passed_ball,p_pickoff_error_1b,p_pickoff_error_2b,p_pickoff_error_3b,p_pitchout,p_relief_no_out,p_sac_bunt,p_sac_fly,p_starting_p,p_swinging_strike,p_unearned_run,p_total_ball,p_total_bases,p_total_caught_stealing,p_total_pickoff,p_total_pickoff_attempt,p_total_pickoff_error,p_total_pitches,p_total_sacrifices,p_total_strike,p_total_swinging_strike,p_inh_runner,p_inh_runner_scored,p_beq_runner,p_beq_runner_scored,p_reached_on_error,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pitch_hand,n,n_ff_formatted,ff_avg_speed,ff_avg_spin,ff_avg_break_x,ff_avg_break_z,ff_avg_break,ff_range_speed,n_sl_formatted,sl_avg_speed,sl_avg_spin,sl_avg_break_x,sl_avg_break_z,sl_avg_break,sl_range_speed,n_ch_formatted,ch_avg_speed,ch_avg_spin,ch_avg_break_x,ch_avg_break_z,ch_avg_break,ch_range_speed,n_cukc_formatted,cu_avg_speed,cu_avg_spin,cu_avg_break_x,cu_avg_break_z,cu_avg_break,cu_range_speed,n_sift_formatted,si_avg_speed,si_avg_spin,si_avg_break_x,si_avg_break_z,si_avg_break,si_range_speed,n_fc_formatted,fc_avg_speed,fc_avg_spin,fc_avg_break_x,fc_avg_break_z,fc_avg_break,fc_range_speed,n_fs_formatted,fs_avg_speed,fs_avg_spin,fs_avg_break_x,fs_avg_break_z,fs_avg_break,fs_range_speed,n_kn_formatted,kn_avg_speed,kn_avg_spin,kn_avg_break_x,kn_avg_break_z,kn_avg_break,kn_range_speed,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break_z,fastball_avg_break,fastball_range_speed,n_breaking_formatted,breaking_avg_speed,breaking_avg_spin,breaking_avg_break_x,breaking_avg_break_z,breaking_avg_break,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm"

def get_custom_leaderboard(endpoint):
    response = requests.get(endpoint)
    soup = BeautifulSoup(response.text, 'html.parser')

    # find leaderboard data (should only be one element.)
    script = str(soup.find_all("div", {"class":"article-template"})[0])

    # use regex and put everything within brackets to create a pseudo-list
    # then load this as json.
    leaderboard = json.loads('[' + re.findall('\[(.*?)\]', script)[0] + ']')

    df = pd.DataFrame(leaderboard)

    return df

batter_custom_leaderboard  = get_custom_leaderboard(batters_endpoint)
pitcher_custom_leaderboard = get_custom_leaderboard(pitcher_endpoint)

In [9]:
import re 
import requests
import pandas as pd
from IPython.display import display, HTML

# team based stats

def get_team_stats():

    endpoints = {
        'batting':"https://www.espn.com/mlb/stats/team",
        'pitching':"https://www.espn.com/mlb/stats/team/_/view/pitching"
    }

    return_dfs = {}

    for k,v in endpoints.items():
        response = requests.get(v)
        soup = BeautifulSoup(response.text, 'html.parser')
        raw_string = re.findall('\"teamStats\":(.*?),\"dictionary\":',str(soup))[0]
        team_stats = json.loads(raw_string)

        stat_list = []

        for team in team_stats:
            team_dict = {
                'abbr':team['team']['abbrev'],
                'name':team['team']['displayName']
            }
            
            for stat in team['stats']:
                try:
                    team_dict[stat['name']] = float(stat['value'])
                except ValueError as err:
                    team_dict[stat['name']] = stat['value']

            stat_list.append(team_dict)

        if k == 'batting':
            df = pd.DataFrame(stat_list).sort_values(by=['OPS'], ascending=True).reset_index(drop=True)
            
            return_dfs['batting'] = df 
        
        else:
            df = pd.DataFrame(stat_list).sort_values(by=['WHIP'], ascending=False).reset_index(drop=True)
            
            return_dfs['pitching'] = df

    return return_dfs

_ = get_team_stats()

In [10]:
import requests
from bs4 import BeautifulSoup
# via requests, first find a list of all the game references to scrape.

endpoints = {
    'probable_pitchers':'https://baseballsavant.mlb.com/probable-pitchers',
    'base'             :'https://baseballsavant.mlb.com',
    'batting':"https://www.espn.com/mlb/stats/team",
    'pitching':"https://www.espn.com/mlb/stats/team/_/view/pitching",
    'batters_percentile':'https://baseballsavant.mlb.com/leaderboard/percentile-rankings?type=batter&team=',
    'pitchers_percentile':'https://baseballsavant.mlb.com/leaderboard/percentile-rankings?type=pitcher&year=2022&team=',
    'batters_custom':"https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&sort=4&sortDir=desc&min=10&selections=player_age,b_ab,b_total_pa,b_total_hits,b_single,b_double,b_triple,b_home_run,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,b_rbi,b_lob,b_total_bases,r_total_caught_stealing,r_total_stolen_base,b_ab_scoring,b_ball,b_called_strike,b_catcher_interf,b_foul,b_foul_tip,b_game,b_gnd_into_dp,b_gnd_into_tp,b_gnd_rule_double,b_hit_by_pitch,b_hit_ground,b_hit_fly,b_hit_into_play,b_hit_line_drive,b_hit_popup,b_out_fly,b_out_ground,b_out_line_drive,b_out_popup,b_intent_ball,b_intent_walk,b_interference,b_pinch_hit,b_pinch_run,b_pitchout,b_played_dh,b_sac_bunt,b_sac_fly,b_swinging_strike,r_caught_stealing_2b,r_caught_stealing_3b,r_caught_stealing_home,r_defensive_indiff,r_interference,r_pickoff_1b,r_pickoff_2b,r_pickoff_3b,r_run,r_stolen_base_2b,r_stolen_base_3b,r_stolen_base_home,b_total_ball,b_total_sacrifices,b_total_strike,b_total_swinging_strike,b_total_pitches,r_stolen_base_pct,r_total_pickoff,b_reached_on_error,b_walkoff,b_reached_on_int,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pop_2b_sba_count,pop_2b_sba,pop_2b_sb,pop_2b_cs,pop_3b_sba_count,pop_3b_sba,pop_3b_sb,pop_3b_cs,exchange_2b_3b_sba,maxeff_arm_2b_3b_sba,n_outs_above_average,n_fieldout_5stars,n_opp_5stars,n_5star_percent,n_fieldout_4stars,n_opp_4stars,n_4star_percent,n_fieldout_4stars,n_fieldout_3stars,n_opp_3stars,n_3star_percent,n_fieldout_3stars,n_opp_2stars,n_2star_percent,n_fieldout_2stars,n_opp_2stars,n_1star_percent,n_fieldout_1stars,n_opp_1stars,n_1star_percent,rel_league_reaction_distance,rel_league_burst_distance,rel_league_routing_distance,rel_league_bootup_distance,f_bootup_distance,n_bolts,hp_to_1b,sprint_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm",
    'pitchers_custom':"https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=pitcher&filter=&sort=4&sortDir=asc&min=10&selections=player_age,p_game,p_formatted_ip,p_total_pa,p_ab,p_total_hits,p_single,p_double,p_triple,p_home_run,p_strikeout,p_walk,p_k_percent,p_bb_percent,batting_avg,slg_percent,on_base_percent,on_base_plus_slg,isolated_power,p_earned_run,p_run,p_save,p_blown_save,p_out,p_win,p_loss,p_wild_pitch,p_balk,p_shutout,p_era,p_opp_batting_avg,p_opp_on_base_avg,p_total_stolen_base,p_pickoff_attempt_1b,p_pickoff_attempt_2b,p_pickoff_attempt_3b,p_pickoff_1b,p_pickoff_2b,p_pickoff_3b,p_lob,p_rbi,p_stolen_base_2b,p_stolen_base_3b,p_stolen_base_home,p_quality_start,p_walkoff,p_run_support,p_ab_scoring,p_automatic_ball,p_ball,p_called_strike,p_catcher_interf,p_caught_stealing_2b,p_caught_stealing_3b,p_caught_stealing_home,p_complete_game,p_defensive_indiff,p_foul,p_foul_tip,p_game_finished,p_game_in_relief,p_gnd_into_dp,p_gnd_into_tp,p_gnd_rule_double,p_hit_by_pitch,p_hit_fly,p_hit_ground,p_hit_line_drive,p_hit_into_play,p_hit_scoring,p_hold,p_intent_ball,p_intent_walk,p_missed_bunt,p_out_fly,p_out_ground,p_out_line_drive,p_passed_ball,p_pickoff_error_1b,p_pickoff_error_2b,p_pickoff_error_3b,p_pitchout,p_relief_no_out,p_sac_bunt,p_sac_fly,p_starting_p,p_swinging_strike,p_unearned_run,p_total_ball,p_total_bases,p_total_caught_stealing,p_total_pickoff,p_total_pickoff_attempt,p_total_pickoff_error,p_total_pitches,p_total_sacrifices,p_total_strike,p_total_swinging_strike,p_inh_runner,p_inh_runner_scored,p_beq_runner,p_beq_runner_scored,p_reached_on_error,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,poorlyunder_percent,poorlytopped_percent,poorlyweak_percent,hard_hit_percent,z_swing_percent,z_swing_miss_percent,oz_swing_percent,oz_swing_miss_percent,oz_contact_percent,out_zone_swing_miss,out_zone_swing,out_zone_percent,out_zone,meatball_swing_percent,meatball_percent,pitch_count_offspeed,pitch_count_fastball,pitch_count_breaking,pitch_count,iz_contact_percent,in_zone_swing_miss,in_zone_swing,in_zone_percent,in_zone,edge_percent,edge,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,batted_ball,f_strike_percent,groundballs_percent,groundballs,flyballs_percent,flyballs,linedrives_percent,linedrives,popups_percent,popups,pitch_hand,n,n_ff_formatted,ff_avg_speed,ff_avg_spin,ff_avg_break_x,ff_avg_break_z,ff_avg_break,ff_range_speed,n_sl_formatted,sl_avg_speed,sl_avg_spin,sl_avg_break_x,sl_avg_break_z,sl_avg_break,sl_range_speed,n_ch_formatted,ch_avg_speed,ch_avg_spin,ch_avg_break_x,ch_avg_break_z,ch_avg_break,ch_range_speed,n_cukc_formatted,cu_avg_speed,cu_avg_spin,cu_avg_break_x,cu_avg_break_z,cu_avg_break,cu_range_speed,n_sift_formatted,si_avg_speed,si_avg_spin,si_avg_break_x,si_avg_break_z,si_avg_break,si_range_speed,n_fc_formatted,fc_avg_speed,fc_avg_spin,fc_avg_break_x,fc_avg_break_z,fc_avg_break,fc_range_speed,n_fs_formatted,fs_avg_speed,fs_avg_spin,fs_avg_break_x,fs_avg_break_z,fs_avg_break,fs_range_speed,n_kn_formatted,kn_avg_speed,kn_avg_spin,kn_avg_break_x,kn_avg_break_z,kn_avg_break,kn_range_speed,n_fastball_formatted,fastball_avg_speed,fastball_avg_spin,fastball_avg_break_x,fastball_avg_break_z,fastball_avg_break,fastball_range_speed,n_breaking_formatted,breaking_avg_speed,breaking_avg_spin,breaking_avg_break_x,breaking_avg_break_z,breaking_avg_break,breaking_range_speed,n_offspeed_formatted,offspeed_avg_speed,offspeed_avg_spin,offspeed_avg_break_x,offspeed_avg_break_z,offspeed_avg_break,offspeed_range_speed,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm"
}

def generate_pitching_matchups(endpoints):

    response = requests.get(endpoints['probable_pitchers'])
    soup = BeautifulSoup(response.text, 'html.parser')

    matchup_strings = [i.text for i in soup.find_all("a", {"class": "matchup-link"}) if 'src=' not in str(i)]

    matchup_strings = [pitcher for pitcher in matchup_strings if pitcher not in ['James Kaprielian',]]

    matchup_tuples = [(matchup_strings[i],matchup_strings[i+1]) for i in range(0,len(matchup_strings),2)]

    df = get_baseball_savant_leaderboard(endpoints['pitchers_percentile'])

    df = df.drop([  'year', 'player_type', 'player_id', 'pos',
                    'percent_speed_order', 'percent_rank_pop_2b', 'percent_rank_oaa',
                    'percent_rank_framing', 'percent_rank_jump',
                    'percent_rank_fastball_velo', 'percent_rank_fastball_spin',
                    'percent_rank_cu_spin','href'], axis = 1)

    df[df.columns[2:]] = df[df.columns[2:]].div(100)

    df = df[['player_name', 'team_name', 'percent_rank_barrel',
       'percent_rank_barrel_batted_rate', 'percent_rank_exit_velocity_avg',
       'percent_rank_exit_velocity_max',
       'percent_rank_xba', 'percent_rank_xslg',
       'percent_rank_hard_hit_percent',
       'percent_rank_xwobacon', 'percent_rank_k_percent', 'percent_rank_bb_percent',
       'percent_rank_whiff_percent', 'percent_rank_chase_percent',
       'percent_rank_ba', 'percent_rank_xbacon',
       'percent_rank_babip', 'percent_rank_obp', 'percent_rank_slg',
       'percent_rank_xobp', 'percent_rank_xiso',
       'percent_rank_xera']]

    # add more weight to strikeouts, hardhit, and bb%
    df['percent_rank_hard_hit_percent'] = df['percent_rank_hard_hit_percent'].mul(1.55)
    df['percent_rank_k_percent'] = df['percent_rank_k_percent'].mul(1.75)
    df['percent_rank_whiff_percent'] = df['percent_rank_whiff_percent'].mul(1.75)
    df['percent_rank_bb_percent'] = df['percent_rank_bb_percent'].mul(2)    
    df['percent_rank_exit_velocity_avg'] = df['percent_rank_exit_velocity_avg'].mul(1.5)
    df['percent_rank_xwobacon'] = df['percent_rank_xwobacon'].mul(1.5)
    
    df['rating'] = df[df.columns[2:]].sum(axis=1).round(2)

    df = df.sort_values(by=['rating'], ascending=False).reset_index(drop=True)

    df['rating'] = (df['rating'] - df['rating'].mean()) / df['rating'].std(ddof=0)

    mismatch_tuples = []
    pitchers_found  = []

    for matchup in matchup_tuples:
        away_pitcher = matchup[0]
        home_pitcher = matchup[1]
        away_rating, home_rating = None, None
        try:
            away_rating = round(df.loc[df['player_name'] == away_pitcher, 'rating'].item(),2)
            pitchers_found.append(away_pitcher)
        except Exception as err:
            print('not enough 2021 data on:',away_pitcher)

        try:
            home_rating = round(df.loc[df['player_name'] == home_pitcher, 'rating'].item(),2)
            pitchers_found.append(home_pitcher)
            
        except Exception as err:
            print('not enough 2021 data on:',home_pitcher)
        
        # try calculate mismatch.
        try:
            
            mismatch = round(away_rating - home_rating,2)
            mismatch_tuples.append((away_pitcher,home_pitcher, mismatch))

        except Exception as err:
            # print(err)
            print(f'mismatch couldn\'t be calculated for {away_pitcher} vs. {home_pitcher}.')

    sorted_available_matches = set(sorted(mismatch_tuples, key=lambda x: x[1]))

    mismatch_df = pd.DataFrame(sorted_available_matches, columns =['away pitcher', 'home pitcher', 'mismatch difference (positive = away favored, negative = home favored.']).sort_values(
        by=['mismatch difference (positive = away favored, negative = home favored.'], ascending=False
    )

    todays_pitchers = df[df['player_name'].isin(pitchers_found)]

    todays_pitchers = todays_pitchers.sort_values(by=['rating'], ascending=False).reset_index(drop=True)

    cols = todays_pitchers.columns.to_list()
    rearranged = cols[0:2] + cols[-1:] + cols[2:-1]

    todays_pitchers = todays_pitchers[rearranged].dropna()

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(mismatch_df)
        display(todays_pitchers)

    return mismatch_df, todays_pitchers

mismatch_df, todays_pitchers = generate_pitching_matchups(endpoints)

not enough 2021 data on: Cade Cavalli
mismatch couldn't be calculated for Cade Cavalli vs. Bryse Wilson.
not enough 2021 data on: JT Chargois
mismatch couldn't be calculated for Chris Bassitt vs. JT Chargois.
not enough 2021 data on: Jose Quintana
mismatch couldn't be calculated for Spencer Strider vs. Jose Quintana.


,away pitcher,home pitcher,"mismatch difference (positive = away favored, negative = home favored."
1,Gerrit Cole,JP Sears,2.25
0,Tyler Anderson,Mike Minor,1.70
6,Mitch White,Chad Kuhl,1.09
5,Shane Bieber,Logan Gilbert,0.94
8,Michael Wacha,Tyler Alexander,0.66
10,Glenn Otto,Kyle Bradish,0.40
4,Alex Wood,Joe Ryan,-0.13
2,Lance McCullers Jr.,Kris Bubic,-0.39
7,Bailey Falter,Reid Detmers,-0.48
3,Justin Steele,Freddy Peralta,-0.53


,player_name,team_name,rating,percent_rank_barrel,percent_rank_barrel_batted_rate,percent_rank_exit_velocity_avg,percent_rank_exit_velocity_max,percent_rank_xba,percent_rank_xslg,percent_rank_hard_hit_percent,percent_rank_xwobacon,percent_rank_k_percent,percent_rank_bb_percent,percent_rank_whiff_percent,percent_rank_chase_percent,percent_rank_ba,percent_rank_xbacon,percent_rank_babip,percent_rank_obp,percent_rank_slg,percent_rank_xobp,percent_rank_xiso,percent_rank_xera
0,Spencer Strider,Braves,1.692683,0.48,0.74,1.020,0.40,0.96,0.95,0.9145,0.990,1.7325,0.62,1.6450,0.57,0.95,0.68,0.52,0.90,0.96,0.91,0.86,0.95
1,Tyler Anderson,Dodgers,1.617088,0.31,0.83,1.455,0.38,0.68,0.67,1.4725,1.335,0.5425,1.76,0.8575,0.93,0.66,0.89,0.77,0.80,0.65,0.84,0.65,0.78
2,Freddy Peralta,Brewers,1.553835,0.93,0.96,1.020,0.48,0.80,0.80,0.9920,1.155,1.3475,0.62,1.4700,0.52,0.77,0.77,0.62,0.68,0.76,0.68,0.73,0.75
3,Chris Bassitt,Mets,1.408815,0.25,0.79,1.440,0.40,0.71,0.69,1.3795,1.200,1.0150,1.48,0.8225,0.25,0.55,0.84,0.55,0.66,0.64,0.79,0.69,0.76
4,Gerrit Cole,Yankees,1.297736,0.09,0.38,0.765,0.27,0.80,0.65,0.8990,0.450,1.6100,1.48,1.5925,0.83,0.75,0.47,0.62,0.87,0.55,0.90,0.39,0.82
5,Justin Steele,Cubs,1.015411,0.56,0.96,1.200,0.31,0.53,0.81,1.3175,1.080,1.1025,0.56,0.5600,0.20,0.43,0.40,0.20,0.37,0.74,0.42,0.94,0.67
6,Shane Bieber,Guardians,0.979927,0.11,0.58,0.240,0.56,0.44,0.55,0.2325,0.720,1.1200,1.72,1.4000,0.73,0.55,0.29,0.39,0.76,0.70,0.74,0.63,0.67
7,Joe Ryan,Twins,0.936730,0.13,0.21,0.780,0.28,0.72,0.47,1.0850,0.885,0.9625,1.20,0.8400,0.42,0.66,0.84,0.80,0.68,0.29,0.74,0.26,0.60
8,Alex Wood,Giants,0.814852,0.25,0.72,0.825,0.31,0.44,0.54,0.6045,0.675,0.9625,1.70,0.8575,0.57,0.31,0.35,0.28,0.49,0.43,0.59,0.60,0.56
9,Johnny Cueto,White Sox,0.811766,0.22,0.72,1.050,0.06,0.28,0.40,1.0230,1.155,0.1050,1.74,0.0175,0.83,0.31,0.75,0.61,0.61,0.48,0.61,0.54,0.53
